In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
fake = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\Fake.csv',encoding='latin1')
true = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\True.csv',encoding='latin1')

In [3]:
fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17


In [5]:
fake['target'] = 'FAKE'
true['target'] = 'REAL'

In [6]:
data = pd.concat([fake, true]).reset_index(drop = True)
data

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-Aug-17,REAL
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-Aug-17,REAL
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-Aug-17,REAL
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-Aug-17,REAL


In [7]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,title,text,subject,date,target
0,South Korea says U.S. designation of North Kor...,SEOUL (Reuters) - South Korea s foreign minist...,worldnews,21-Nov-17,REAL
1,Trump Refuses To Say âRadical Islamic Terro...,Donald Trump is a complete hypocrite.Just a da...,News,"May 21, 2017",FAKE
2,NATIONAL PET CHAIN Linked To Outbreak of Bacte...,The CDC is investigating an outbreak of a bact...,politics,"Sep 14, 2017",FAKE
3,Violence flares at protest near U.S. Embassy i...,BEIRUT (Reuters) - Lebanese security forces fi...,worldnews,10-Dec-17,REAL
4,"Pope, Jordan's King Abdullah, discuss Trump's ...",VATICAN CITY (Reuters) - Pope Francis and Jord...,worldnews,19-Dec-17,REAL


In [8]:
data.drop(["date"],axis=1,inplace=True)
data.drop(["title"],axis=1,inplace=True)
data

,text,subject,target
0,SEOUL (Reuters) - South Korea s foreign minist...,worldnews,REAL
1,Donald Trump is a complete hypocrite.Just a da...,News,FAKE
2,The CDC is investigating an outbreak of a bact...,politics,FAKE
3,BEIRUT (Reuters) - Lebanese security forces fi...,worldnews,REAL
4,VATICAN CITY (Reuters) - Pope Francis and Jord...,worldnews,REAL
...,...,...,...
44893,JERUSALEM (Reuters) - Israeli forces on Sunday...,worldnews,REAL
44894,Patrick Henningsen 21st Century Wire As 21WIRE...,US_News,FAKE
44895,Like a spoiled child on a spending spree with ...,left-news,FAKE
44896,"RALEIGH, N.C. (Reuters) - North Carolinaâs R...",politicsNews,REAL


In [9]:
data.isnull().any()

text       False
subject    False
target     False
dtype: bool

In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,text,subject,target
0,seoul (reuters) - south korea s foreign minist...,worldnews,REAL
1,donald trump is a complete hypocrite.just a da...,News,FAKE
2,the cdc is investigating an outbreak of a bact...,politics,FAKE
3,beirut (reuters) - lebanese security forces fi...,worldnews,REAL
4,vatican city (reuters) - pope francis and jord...,worldnews,REAL
...,...,...,...
44893,jerusalem (reuters) - israeli forces on sunday...,worldnews,REAL
44894,patrick henningsen 21st century wire as 21wire...,US_News,FAKE
44895,like a spoiled child on a spending spree with ...,left-news,FAKE
44896,"raleigh, n.c. (reuters) - north carolinaâs r...",politicsNews,REAL


In [11]:
import string
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
data['text'] = data['text'].apply(punctuation_removal)
data

,text,subject,target
0,seoul reuters south korea s foreign ministry ...,worldnews,REAL
1,donald trump is a complete hypocritejust a day...,News,FAKE
2,the cdc is investigating an outbreak of a bact...,politics,FAKE
3,beirut reuters lebanese security forces fired...,worldnews,REAL
4,vatican city reuters pope francis and jordan ...,worldnews,REAL
...,...,...,...
44893,jerusalem reuters israeli forces on sunday de...,worldnews,REAL
44894,patrick henningsen 21st century wire as 21wire...,US_News,FAKE
44895,like a spoiled child on a spending spree with ...,left-news,FAKE
44896,raleigh nc reuters north carolinaâs republi...,politicsNews,REAL


In [12]:
#stopwords removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head(50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HZT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,subject,target
0,seoul reuters south korea foreign ministry sai...,worldnews,REAL
1,donald trump complete hypocritejust day curtsy...,News,FAKE
2,cdc investigating outbreak bacterial infection...,politics,FAKE
3,beirut reuters lebanese security forces fired ...,worldnews,REAL
4,vatican city reuters pope francis jordan king ...,worldnews,REAL
5,apparently black lives matter terror group man...,politics,FAKE
6,,politics,FAKE
7,paris reuters investigators friday reconstruct...,worldnews,REAL
8,much promise transparency obama hundreds lawye...,Government News,FAKE
9,ronald reagan largely seen messiah republican ...,News,FAKE


In [18]:
x_train,x_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2)

In [19]:
#tfidf_vectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('dtmodel', DecisionTreeClassifier())])

In [21]:
modeldt = pipeline.fit(x_train,y_train)

In [22]:
prediction = modeldt.predict(x_test)
print("Accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

Accuracy: 99.61%


In [24]:
pred = modeldt.predict(x_test)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        FAKE       1.00      1.00      1.00      4628
        REAL       1.00      1.00      1.00      4352

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [26]:
print(confusion_matrix(y_test, pred))

[[4611   17]
 [  18 4334]]


In [27]:
pickle.dump(modeldt, open('modeldt.pkl','wb'))  
modeldt = pickle.load(open('modeldt.pkl','rb'))

In [28]:
print(modeldt.predict(["How elite leftists celebrate America s Independence Day The Kennedy clan gathered at their Hyannis Port compound on Cape Cod over the weekend for their annual Fourth of July festivities, and took some time to attack Donald Trump.Robert F. Kennedy Jr. s daughter Kathleen, between known as Kick, posted a photos of a pinata of The Donald from a family party over the weekend. It s yuge party!,  wrote Kick in the caption of the Instagram post, which also showed some of her family members milling about in the background.She later deleted the Instagram post just before 11am on Monday.Via: Daily Mail"]))

['FAKE']
